In [1]:
import torch
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import os
import torch
from torch import nn
from torch import nn, optim

import torch.nn.functional as F

%matplotlib inline

In [93]:
# loans =  pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Loans.csv")
# equiv = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Synonyms.csv")
# random = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Randoms.csv")
# hard_neg = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Hard-Negatives.csv")

import pandas as pd
import io
import requests
import csv
 
# s=requests.get(url).content
# c=pd.read_csv(io.StringIO(s.decode('utf-8')))
#equiv = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Synonyms.csv")
# rand = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Randoms.csv")
# train = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/train_final.csv")
# test =pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/test_final.csv")
url_syn = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Synonyms/Hindi-Persian-Synonyms.csv'
url_rand = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Randoms/Hindi-Persian-Randoms.csv'
url_loans = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Loans/Hindi-Persian-Loans.csv'
url_hard = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Hard-Negatives/Hindi-Persian-Hard-Negatives.csv'

url_syn = requests.get(url_syn).content
url_rand = requests.get(url_rand).content
url_loans = requests.get(url_loans).content
url_hard = requests.get(url_hard).content
#c = pd.read_csv(s)
#df_1 = pd.read_csv(url, header=None, sep='\n')
equiv=pd.read_csv(io.StringIO(url_syn.decode('utf-8')))
random = pd.read_csv(io.StringIO(url_rand.decode('utf-8')))
loans=pd.read_csv(io.StringIO(url_loans.decode('utf-8')))
hard_neg = pd.read_csv(io.StringIO(url_hard.decode('utf-8')))





In [126]:
equiv.shape

(6206, 14)

In [136]:
loans.shape,equiv.shape,random.shape, hard_neg.shape

((1344, 14), (6206, 14), (1340, 14), (4282, 14))

In [100]:
#first shuffle all the different sets before creating the splits:
loans = loans.iloc[np.random.permutation(len(loans))]
equiv = equiv.iloc[np.random.permutation(len(equiv))]
random = random.iloc[np.random.permutation(len(random))]
hard_neg = hard_neg.iloc[np.random.permutation(len(hard_neg))]

In [101]:
loans.shape,equiv.shape,random.shape, hard_neg.shape

((1344, 14), (6206, 14), (1340, 14), (4282, 14))

In [127]:
#train sets for previous versions of datasets

# loans_train = loans.iloc[:1200, :]
# loans_test = loans.iloc[1200:-1, :]
# equiv_train = equiv.iloc[:1200, :]
# equiv_test = equiv.iloc[1200:1340, :]
 
# random_train = random.iloc[:1200, :]
# random_test = random.iloc[1200:1340, :]
# hard_neg_train = hard_neg.iloc[:800, :]
# hard_neg_test = hard_neg.iloc[800:880, :]

# version one real distribution : train sets for new versions with updated datasets, version one with 30 percent loan, 70 non loan, and 1/7 hard neg
# loans_train = loans.iloc[:1200, :]
# loans_test = loans.iloc[1200:-1, :]
# equiv_train = equiv.iloc[:1400, :]
# equiv_test = equiv.iloc[1400:1540, :]
 
# random_train = random.iloc[:1200, :]
# random_test = random.iloc[1200:-1, :]
# hard_neg_train = hard_neg.iloc[:400, :]
# hard_neg_test = hard_neg.iloc[400:440, :]

#version two balanced: train sets for new versions with updated datasets, version two with 50 percent balanced loans and rest: 1/7 hard neg, 4/7 synonyms, 2/7 random pairs i.e 
#for train, out of 1200 non-loans(1200 loans) 172 are hard neg, 686 synonyms, 349 random pairs 


# loans_train = loans.iloc[:1200, :]
# loans_test = loans.iloc[1200:-1, :]
# equiv_train = equiv.iloc[:686, :]  #686 syns for train 
# equiv_test = equiv.iloc[1400:1482, :] #82 syns for test
 
# random_train = random.iloc[:349, :] # 349 randos for train 
# random_test = random.iloc[1200:1242, :] #42 random for test 
# hard_neg_train = hard_neg.iloc[:172, :] # 172 hard negs for train 
# hard_neg_test = hard_neg.iloc[172:192, :] #20 hard negs for test

#version three : using the entire dataset, overweighted against loan words

loans_train = loans.iloc[:1200, :] 
loans_test = loans.iloc[1200:-1, :]
equiv_train = equiv.iloc[:5585, :]   #5585 train, 621 test 
equiv_test = equiv.iloc[5585:-1, :]   
 
random_train = random.iloc[:1200, :]
random_test = random.iloc[1200:-1, :]
hard_neg_train = hard_neg.iloc[:3854, :] #3854 , 428 test 
hard_neg_test = hard_neg.iloc[3854:-1, :]








In [137]:
loans_train.shape, loans_test.shape, equiv_train.shape, equiv_test.shape,random_train.shape, random_test.shape,hard_neg_train.shape,hard_neg_test.shape

((1200, 15),
 (143, 15),
 (5585, 15),
 (620, 15),
 (1200, 15),
 (139, 15),
 (3854, 15),
 (427, 15))

In [129]:
loans_train.shape, loans_test.shape, equiv_train.shape, equiv_test.shape,random_train.shape, random_test.shape, hard_neg_train.shape, hard_neg_test.shape
 

((1200, 14),
 (143, 14),
 (5585, 14),
 (620, 14),
 (1200, 14),
 (139, 14),
 (3854, 14),
 (427, 14))

In [105]:
#random.loc[random['loan_word']=='#NAME?']
# equiv_train.loc[equiv_train['loan_word']=='#NAME?']
# hard_neg_test.loc[hard_neg_test['loan_word']=='#NAME?'] 
# loans.loc[loans['loan_word']=='#NAME?']



In [106]:
# DON't DO THIS FOR NEW Datasets , drop the wiktionary IPA since we are using Epitran IPAs for all sets

# random_train = random_train.drop(["original_word_ipas", "loan_word_ipas"], axis=1)
# random_test = random_test.drop(["original_word_ipas", "loan_word_ipas"], axis=1)
# hard_neg_train = hard_neg_train.drop(["original_word_ipas", "loan_word_ipas"], axis=1)
# hard_neg_test = hard_neg_test.drop(["original_word_ipas", "loan_word_ipas"], axis=1)


In [130]:
#put binary labels for loan and non loan words

loans_train['label_bin'] =1
loans_test['label_bin']=1
equiv_train['label_bin']=0
equiv_test['label_bin']=0
random_train['label_bin']=0
random_test['label_bin']=0
hard_neg_train['label_bin']=0
hard_neg_test['label_bin']=0



/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/s/chopin/d/proj/ramfis-aida/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is tryin

In [131]:
loans_train.shape, equiv_train.shape,random_train.shape,hard_neg_train.shape

((1200, 15), (5585, 15), (1200, 15), (3854, 15))

In [132]:
train_final = pd.concat([loans_train, equiv_train,random_train,hard_neg_train ])
test_final   = pd.concat([loans_test, equiv_test,random_test,hard_neg_test ])
        

In [133]:
#shuffle it after final concatenation 

train_final = train_final.iloc[np.random.permutation(len(train_final))]
test_final = test_final.iloc[np.random.permutation(len(test_final))]

In [134]:
train_final.shape, test_final.shape

((11839, 15), (1329, 15))

In [135]:
train_final.to_csv('train_final_production_alldata.csv')
test_final.to_csv('test_final_production_alldata.csv')

In [123]:
test_final

,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin
1296,हद,حد,ɦəd,hd,To the extent,Hadd,0.666667,0.333333,0.312500,0.347222,2.500000,0.347222,2,loan,1
1129,वापसी,واپسی,vaːpsiː,vɒpsj,Return,Returns,0.571429,0.200000,0.050000,0.058333,0.875000,0.050000,5,loan,1
1117,वफ़ादार,وفادار,vəfaːdaːr,vfɒdɒr,faithful,Loyal,0.555556,0.142857,0.163690,0.178571,1.464286,0.178571,7,loan,1
1046,रफ़ू,نگه,rəfuː,ŋh,darn,Hold,1.000000,1.000000,0.531250,0.614583,5.312500,0.598958,4,random,0
6019,रोशनी,تغییر می کند,roʃniː,tɣjjr mj knd,Light,Changes,0.916667,0.800000,0.525000,0.587500,5.175000,0.575000,12,synonym,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,बल्कि,بلکه,bəlki,blkh,Rather,But,0.400000,0.400000,0.220833,0.250000,2.475000,0.241667,5,loan,1
154,क़तार,قطار,qətaːr,ɣtɒr,queue,Train,0.666667,0.200000,0.229167,0.250000,2.100000,0.250000,5,loan,1
309,ग़ज़ा,دامن,ɣəzaː,dɒmn,Gaza,Skirt,1.000000,0.750000,0.234375,0.260417,4.125000,0.234375,5,random,0
1685,ताजपोशी,تاب‌بازی,taːd͡ʒpoʃiː,tɒb‌bɒzj,coronation,Swinging,0.909091,0.285714,0.092262,0.101190,1.178571,0.092262,8,hard_negative,0


In [18]:
equiv.sample(n = 20, random_state = 3)

,loan_word,original_word,loan_word_epitran,original_word_epitran,Fast Levenshtein,Dolgo Prime Distance,Feature Edit Distance,Hamming Feature Distance,Weighted Feature Distance,Fast Levenshtein Distance Div Maxlen
7396,शानदार,عالی,ʃaːndaːr,ʔɒlj,1.000000,0.500000,0.304688,0.343750,3.468750,1.000000
6185,परंपरागत,مرسوم,pərəmpraːɡət,mrsvm,0.833333,0.750000,0.503472,0.565972,4.614583,0.833333
5764,राक्षस,غول,raːkʂəs,ɣvl,1.000000,0.714286,0.434524,0.494048,3.964286,1.000000
5425,लाभदायक,بارور,laːb̤daːjək,bɒrvr,0.909091,0.454545,0.337121,0.386364,3.602273,0.909091
5190,साफदिल,آدم ساده,saːpʰdil,ɒdm sɒdh,1.000000,0.500000,0.273438,0.302083,3.750000,1.000000
5227,अभेद्य,تسخیر ناپذیر,ab̤edjə,tsxjr nɒپzjr,0.916667,0.583333,0.366319,0.420139,3.729167,0.916667
6710,थकाऊ,معمولی,tʰəkaːuː,mʔmvlj,1.000000,0.750000,0.281250,0.317708,3.750000,1.000000
3169,मूल्य कम करना,ناسزا,muːljə kəmə kərnaː,nɒszɒ,1.000000,0.666667,0.480324,0.537037,4.180556,1.000000
1285,हताश,گیج شده,ɦətaːʃ,ɡjd͡ʒ ʃdh,0.888889,0.444444,0.203704,0.226852,2.930556,0.888889
577,हाय,فاجعه,ɦaːj,fɒd͡ʒʔh,1.000000,0.428571,0.321429,0.351190,3.053571,1.000000


In [2]:
equiv_words = pd.read_csv("Equivalent.csv")

In [3]:
equiv_words

,loan_word,original_word,loan_word_epitran,original_word_epitran,Fast Levenshtein,Dolgo Prime Distance,Feature Edit Distance,Hamming Feature Distance,Weighted Feature Distance,Fast Levenshtein Distance Div Maxlen
0,छोड़ देना,دست برداشتن از,t͡ʃʰoɽə denaː,dst brdɒʃtn ɒz,0.928571,0.500000,0.352679,0.383929,3.750000,0.928571
1,छोड़ देना,رد کردن,t͡ʃʰoɽə denaː,rd krdn,0.846154,0.461538,0.232372,0.275641,3.048077,0.846154
2,छोड़ देना,دست بردارید,t͡ʃʰoɽə denaː,dst brdɒrjd,0.923077,0.615385,0.258013,0.301282,3.317308,0.923077
3,छोड़ देना,رها کردن,t͡ʃʰoɽə denaː,rhɒ krdn,0.846154,0.461538,0.190705,0.230769,2.615385,0.846154
4,छोड़ देना,کویر,t͡ʃʰoɽə denaː,kvjr,1.000000,0.461538,0.323718,0.375000,3.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
8445,उमंग,اشتیاق,uməŋɡə,ɒʃtjɒɣ,1.000000,0.833333,0.250000,0.277778,5.270833,1.000000
8446,परोक्ष,مورب,pərokʂə,mvrb,0.857143,0.857143,0.473214,0.523810,4.892857,0.857143
8447,परोक्ष,خودسر,pərokʂə,xvdsr,1.000000,0.857143,0.392857,0.464286,4.375000,1.000000
8448,स्वच्छंद,مورب,svət͡ʃt͡ʃʰəndə,mvrb,0.928571,0.571429,0.361607,0.404762,3.196429,0.928571


In [4]:
hardneg_words = pd.read_csv("Epi_Hard_Neg.csv")

In [5]:
hardneg_words

,loan_word,original_word,loan_word_epitran,original_word_epitran,label,Fast Levenshtein,Dolgo Prime Distance,Feature Edit Distance,Hamming Feature Distance,Weighted Feature Distance,Fast Levenshtein Distance Div Maxlen
0,बंदोबस्त,آبدست,aːbdəstə,ɒbdst,0,0.500000,0.250000,0.239583,0.265625,2.000000,0.500000
1,साज़िश,سازش,saːziʃ,sɒzʃ,1,0.500000,0.166667,0.170139,0.187500,1.458333,0.500000
2,शिकार,دیدار,diːdaːr,djdɒr,0,0.571429,0.142857,0.035714,0.041667,0.625000,0.571429
3,अदा,کدو,kədduː,kdv,0,0.666667,0.500000,0.326389,0.368056,2.791667,0.666667
4,देर,دیر,der,djr,1,0.333333,0.333333,0.041667,0.055556,0.958333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...
12079,शर्बत,شربت,ʃərbət,ʃrbt,1,0.333333,0.333333,0.298611,0.333333,2.416667,0.333333
12080,नवीद,خود,xud,xvd,0,0.333333,0.333333,0.104167,0.125000,2.666667,0.333333
12081,बेवफ़ा,بی‌وفا,bevfaː,bj‌vfɒ,1,0.666667,0.166667,0.041667,0.048611,0.729167,0.666667
12082,आब,او,o,ɒv,0,1.000000,0.500000,0.479167,0.520833,3.875000,1.000000


In [7]:
sum(hardneg_words['label']==0)

10740

In [10]:
%store -r Epi_result

In [11]:
Epi_result

,loan_word,original_word,loan_word_epitran,original_word_epitran,fast levenshtein,Dolgo prime distance,Feature edit distance,Hamming feature distance,Weighted feature distance,Fast Levenshtein distance div maxlen,label,features_loan
0,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
1,ख़राश,طرفدار,xəraːʃ,trfdɒr,5,6,1.916667,2.333333,16.5625,1.000000,0,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, ..."
2,चिराग़,نقاب,t͡ʃiraːɣ,nɣɒb,4,8,1.625000,1.833333,13.5625,1.000000,0,"[-1, -1, 1, -1, 1, -1, -1, 0, -1, -1, -1, -1, ..."
3,मरहम,شوهر,mərɦəm,ʃvhr,4,6,2.625000,3.083333,21.2500,1.000000,0,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, -1,..."
4,हैवानियत,خُوشنَوِیسی,ɦæːvaːnijət,xُvʃnowejsj,5,10,2.833333,3.125000,25.7500,0.909091,0,"[-1, 1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -1..."
...,...,...,...,...,...,...,...,...,...,...,...,...
14779,तहसील,تحصیل,təɦsiːl,thsjl,2,4,1.062500,1.208333,8.8125,0.571429,1,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
14780,तीर,برفی,tiːr,brfj,3,4,1.562500,1.916667,13.2500,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
14781,सितारा,قید,sitaːraː,ɣjd,5,8,2.979167,3.416667,24.2500,1.000000,0,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1..."
14782,सबूत,کبابه چینی,səbuːt,kbɒbh t͡ʃjnj,7,10,4.291667,4.750000,34.3750,0.833333,0,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1..."


In [14]:
Epi_result.loc[Epi_result['loan_word']=='तसल्ली']

,loan_word,original_word,loan_word_epitran,original_word_epitran,fast levenshtein,Dolgo prime distance,Feature edit distance,Hamming feature distance,Weighted feature distance,Fast Levenshtein distance div maxlen,label,features_loan
0,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
2066,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
2760,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
5301,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
8151,तसल्ली,تسلی,təslliː,tslj,2,4,1.937500,2.166667,15.9375,0.571429,1,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
8276,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
11586,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
11624,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
12952,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
13366,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
